## generate questions by openai

In [36]:
import openai 
from openai import OpenAI

In [39]:
openai_client = OpenAI(api_key = OPENAI_API_KEY)

## step1: create unique ids for each questions

In [ ]:
import requests
import hashlib
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

In [18]:
def generate_unique_question_id(doc):
    by_id = f'{doc['course']} - {doc['question']} - {doc['text'][:10]}'
    id = hashlib.md5(by_id.encode('utf-8')).hexdigest()
    return id[:10]

In [19]:
for doc in documents:
    doc['id'] = generate_unique_question_id(doc)

In [ ]:
from  collections import defaultdict
grouped = defaultdict(list)
for d in documents:
    grouped[d['id']].append(d)

## step2: create a prompt

In [43]:
prompt_template = ''' 
You are a student. Based on the record below, generate 5 new user questions that are as short as possible. 

Record:
section: {section}
question: {question}
answer: {text}

Return only a JSON array of the questions, without any code blocks or extra text, in this format:
['question1', 'question2', 'question3', 'question4', 'question5']
'''.strip()

In [ ]:
openai_client.chat.completions()

In [54]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages = [{'role': 'user',
                    'content': prompt}]
        )
    json_response = response.choices[0].message.content
    return json_response

In [55]:
generate_questions(documents[0])

"['What is the course start date?', 'What time does the course begin?', 'How do I register for the course?', 'Where can I find course announcements?', 'How do I join the Telegram channel?']"